In [ ]:
import pandas as pd
import subprocess
import time
from tqdm import tqdm
import os

# =========================
# CẤU HÌNH
# =========================
INPUT_FILE = "movies_processing.csv"
OUTPUT_FILE = "movies_labeled_ollama.csv"

GENRES = [
    "Documentary",
    "Comedy",
    "Action",
    "Thriller",
    "Romance",
    "Horror",
    "Short",
    "Crime",
    "Adventure",
    "Animation"
]

START_FROM_ROW = 2000   # resume nếu cần
MODEL_NAME = "llama3"

# =========================
# HÀM GỌI OLLAMA
# =========================
def classify_movie(plot):
    prompt = f"""
You are a movie expert.

Classify the following movie plot into EXACTLY ONE genre from this list:
{", ".join(GENRES)}

Rules:
- Return ONLY the genre name
- No explanation
- Choose the most dominant genre

Plot:
\"\"\"{plot}\"\"\"
"""

    try:
        result = subprocess.run(
            ["ollama", "run", MODEL_NAME],
            input=prompt,
            capture_output=True,
            text=True,
            encoding="utf-8",     # 👈 QUAN TRỌNG
            errors="ignore",      # 👈 QUAN TRỌNG
            timeout=60
        )


        label = result.stdout.strip()

        # Chuẩn hóa output
        for g in GENRES:
            if g.lower() == label.lower():
                return g

        return "Unknown"

    except Except ion as e:
        print("Lỗi:", e)
        return "Error"

# =========================
# MAIN
# =========================
def main():
    if os.path.exists(OUTPUT_FILE):
        print("🔁 Resume từ file cũ...")
        df = pd.read_csv(OUTPUT_FILE)
    else:
        df = pd.read_csv(INPUT_FILE)

    if "genre_ollama" not in df.columns:
        df["genre_ollama"] = ""

    df["genre_ollama"] = df["genre_ollama"].astype(str)

    for idx, row in tqdm(df.iterrows(), total=len(df)):

        if idx < START_FROM_ROW:
            continue

        if row["genre_ollama"].strip() not in ["", "nan", "Error"]:
            continue

        plot = str(row["Plot_clean"])

        if plot.strip() == "" or plot.lower() == "nan":
            df.at[idx, "genre_ollama"] = "Unknown"
            continue

        label = classify_movie(plot)
        df.at[idx, "genre_ollama"] = label

        tqdm.write(f"{idx:<5} | {label}")

        # Lưu tạm mỗi 20 dòng
        if idx % 20 == 0:
            df.to_csv(OUTPUT_FILE, index=False)

        time.sleep(0.3)  # tránh CPU 100%

    df.to_csv(OUTPUT_FILE, index=False)
    print("\n✅ HOÀN TẤT! Đã lưu:", OUTPUT_FILE)

if __name__ == "__main__":
    main()


🔁 Resume từ file cũ...


  0%|          | 0/7444 [00:02<?, ?it/s]

2000  | Crime


 27%|██▋       | 2001/7444 [00:05<00:06, 811.56it/s]

2001  | Comedy


...

 34%|███▎      | 2512/7444 [1:45:20<15:12:17, 11.10s/it]

2512  | Thriller


 34%|███▍      | 2513/7444 [1:45:27<18:39:17, 13.62s/it]

2513  | Adventure


 34%|███▍      | 2514/7444 [1:45:28<16:00:43, 11.69s/it]